In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Rectal_Cancer/GSE109057'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome analysis by expression microarrays of rectal cancer derived from the Cancer Institute Hospital of Japanese Foundation for Cancer Research"
!Series_summary	"We performed the expression microrarray experiments for mRNA of rectal cancer tissue."
!Series_overall_design	"Messenger RNA were extracted from primary tumor of 90 rectal cancer patients, hybridized and scanned with Affymetrix PrimeView Human Gene Expression Array. There are 91 samples because one sample was hybridized twice, once in each batch, and is represented by GSM2928780 and GSM2928794."
Sample Characteristics Dictionary:
{0: ['tissue: rectal cancer'], 1: ['Sex: M', 'Sex: F'], 2: ['age: 35 <= age < 40', 'age: 30 <= age < 35', 'age: 55 <= age < 60', 'age: 45 <= age < 50', 'age: 65 <= age < 70', 'age: 75 <= age < 80', 'age: 70 <= age < 75', 'age: 60 <= age < 65', 'age: 50 <= age < 55', 'age: 80 <= age < 85', 'age: 40 <= age < 45'], 3: ['batch: Batch 1', 'batch: Batch 2']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data
if "mRNA" in "!Series_summary\t\"We performed the expression microrarray experiments for mRNA of rectal cancer tissue.\"":
    is_gene_available = True
else:
    is_gene_available = False

# Define the keys for data rows from the sample characteristics dictionary
trait_row = 0  # Recorded in key 0 since all samples are rectal cancer
age_row = 2  # Recorded in key 2
gender_row = 1  # Recorded in key 1

# Define data type and conversion functions

# Function to convert trait
def convert_trait(value):
    return 1  # Placeholder function to return 1 for rectal cancer

# Function to convert age
def convert_age(value):
    try:
        age_range = value.split(': ')[1]  # Extract age range block
        mid_age = sum(map(int, age_range.split(' <= age < '))) / 2  # Take the average of age range
        return mid_age
    except:
        return None

# Function to convert gender
def convert_gender(value):
    try:
        gender = value.split(': ')[1]
        return 1 if gender == 'M' else 0  # Male -> 1, Female -> 0
    except:
        return None

# Save metadata
save_cohort_info('GSE109057', './preprocessed/Rectal_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Rectal_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Rectal_Cancer/trait_data/GSE109057.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2928730': [1.0, 37.5, 1.0], 'GSM2928731': [1.0, 32.5, 1.0], 'GSM2928732': [1.0, 57.5, 1.0], 'GSM2928733': [1.0, 47.5, 1.0], 'GSM2928734': [1.0, 67.5, 0.0], 'GSM2928735': [1.0, 77.5, 1.0], 'GSM2928736': [1.0, 57.5, 1.0], 'GSM2928737': [1.0, 47.5, 1.0], 'GSM2928738': [1.0, 72.5, 1.0], 'GSM2928739': [1.0, 62.5, 1.0], 'GSM2928740': [1.0, 52.5, 1.0], 'GSM2928741': [1.0, 32.5, 1.0], 'GSM2928742': [1.0, 67.5, 1.0], 'GSM2928743': [1.0, 47.5, 1.0], 'GSM2928744': [1.0, 67.5, 1.0], 'GSM2928745': [1.0, 57.5, 1.0], 'GSM2928746': [1.0, 47.5, 1.0], 'GSM2928747': [1.0, 47.5, 1.0], 'GSM2928748': [1.0, 72.5, 1.0], 'GSM2928749': [1.0, 67.5, 1.0], 'GSM2928750': [1.0, 47.5, 1.0], 'GSM2928751': [1.0, 67.5, 1.0], 'GSM2928752': [1.0, 67.5, 0.0], 'GSM2928753': [1.0, 57.5, 1.0], 'GSM2928754': [1.0, 52.5, 1.0], 'GSM2928755': [1.0, 67.5, 1.0], 'GSM2928756': [1.0, 47.5, 1.0], 'GSM2928757': [1.0, 62.5, 1.0], 'GSM2928758': [1.0, 62.5, 1.0], 'GSM2928759': [1.0, 67.5, 0.0], 'GSM2928760': [1.0, 62.5, 1.0], 'GSM292

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array', 'Human Genome PrimeView Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16', '30-Mar-16'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /

### Step 6: Gene Identifier Mapping

In [7]:
if requires_gene_mapping:
    # 1. Determine the keys for identifiers and gene symbols
    identifier_key = 'ID'
    gene_symbol_key = 'Gene Symbol'

    # 2. Get the dataframe for mapping between probe IDs and genes
    mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

    # 3. Apply the mapping to get the gene expression dataframe
    gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Rectal_Cancer/gene_data/GSE109057.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Rectal_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE109057', './preprocessed/Rectal_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Rectal_Cancer/GSE109057.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Rectal_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Rectal_Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 52.5
  50% (Median): 62.5
  75%: 67.5
Min: 32.5
Max: 82.5
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 27 occurrences. This represents 29.67% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

